In [1]:
#ALWAYS RUN FIRST!

#Import libraries and packages. Don't worry about the warning if running it on windows, so far not hit an issue. (yn)

import re
from pylab import *
import csv
import psycopg2
import spacy
spacy.load('en')
from spacy.lang.en import English
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
from datetime import datetime
import pickle
import gensim
import os

#Set the names of the files in which you want to save data, these will all be in the data file.

people_data_name = 'people_data'
department_data_name = 'department_names'
meeting_data_name = 'daisy_wheel_meetings'

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Extract the data from the CRM. Don't run if you have an up to date copy of the CRM.

try:
    
    #Opens connection to the CRM asks for peoples name and description.
    #Output rows for row in rows row[0] - first name, row[1] - second name, row[2] - description.
    
    print("Trying to access CRM CSaP database ...")
    
    conn = psycopg2.connect(SERVER_INFO)
    
    cur = conn.cursor()
    
    cur.execute("""SELECT
    person.first_name,
    person.last_name,
    organization.name
    FROM people_person as person
    INNER JOIN
    organizations_personorganizationrole
    ON
    organizations_personorganizationrole.person_id = person.id
    INNER JOIN
    organizations_organization as organization
    ON
    organization.id = organizations_personorganizationrole.organization_id
    ;
    """)
    rows = cur.fetchall()

    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + people_data_name +'.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([r.encode('utf-8') for r in row])
    
    cur.execute("""SELECT
    people_policyfellowshipschedule.date,
    fellow.first_name,
    fellow.last_name,
    academic.first_name,
    academic.last_name
    FROM people_policyfellowshipschedule_people
    JOIN people_person as academic ON people_policyfellowshipschedule_people.person_id = academic.id
    JOIN people_policyfellowshipschedule ON people_policyfellowshipschedule_people.policyfellowshipschedule_id = people_policyfellowshipschedule.id
    JOIN people_policyfellowship ON people_policyfellowshipschedule.policy_fellowship_id = people_policyfellowship.id
    JOIN people_person as fellow ON people_policyfellowship.policy_fellow_id = fellow.id
    ;
    """)
    
    rows = cur.fetchall()
    
    #Saves data to the file called above.
    
    with open(os.getcwd() + '\data\\' + meeting_data_name +'.csv','w+',newline ='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for row in rows:
            wr.writerow([str(row[0]), row[1].encode('utf-8'), row[2].encode('utf-8'), row[3].encode('utf-8'), row[4].encode('utf-8')])
    print("... data downloaded and saved to disk.")
    
except:
    
    #If server isn't online this collects data from the save file.
    
    print("... can't access server, is the tunnel set up? Can continue on previously saved data.")


Trying to access CRM CSaP database ...
... data downloaded and saved to disk.


In [3]:
#Functions for cleaning the text and getting it ready to be procced.

#Gets rid of HTML tags and end of line markers.
#Input: String of text from CRM or internet.
#Output: Cleaned up string of text without HTML tags or end of line markers.

def clean_text(text):
    
    #Removes HTML tags.
    
    clean = re.compile('<.*?>')
    temp_text = re.sub(clean, '', text)
    
    #Removes rouge utf-8 code.
    
    clean = re.compile('\\\\x\w\w')
    temp_text = re.sub(clean, '', temp_text)
    
    clean = re.compile('\\\\x\w')
    temp_text = re.sub(clean, '', temp_text)
    
    #Removes end of line indicators and other junk.
    
    tags = ['\\r','\\n','/','\\t','\\']
    
    for tag in tags:
        temp_text = temp_text.replace(tag,'')
    
    return temp_text

#Tokenizes text, seperates it into a string of words and grammar.
#Input: A string of text.
#Output: A list of words and grammar in order all in lower case.

parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

#Lemmatiser, this finds the root word (i.e. depluralises).
#Input: a token, i.e. a single word or grammar.
#Output: a lemma which is the base of the word and association 

def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word)
    
#Prepares text for the analysis, tokenizes texts, gets rid of words length less than 4 and filters out non-useful words then
#Lemmatisers the text.
#Input: A string of text you want to analysis.
#Output: A list of Lemmas of the meaningful words.

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [tok[0] for tok in nltk.pos_tag(tokens) if tok[1][0] == 'N']
    tokens = [token for token in tokens if len(token) > 4]
    en_stop = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

#Specialised version of the above function for organisations.
#Input: A string of text you want to analysis.
#Output: A list of Lemmas of the meaningful words.

def perpare_organisation(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
    en_stop = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if (token not in en_stop)]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [4]:
#This is imported as a dictionary with the names of the departments as lables and further dictionary as objects linked.
#This uploads the dictionary data into the program from the file, cleaning the data whilst it does it.

def read_dictionary_file(file_name):
    
    print("Trying to load up dictionary ", file_name, " ...")

    new_dictionary = {}

    try:
        with open(os.getcwd() + '\data\\' + file_name +'.csv', 'r') as csvfile:
            dump = list(csv.reader(csvfile))
            dic_place = []
            for row in dump:
                new_name = clean_text(row[0][2:-1])
                cur_dic = new_dictionary
                if new_name == 'END_DIC':
                    del dic_place[len(dic_place) - 1]
                else:
                    for name in dic_place:
                        cur_dic = cur_dic[name]
                    cur_dic[new_name] = {}
                    dic_place.append(new_name)

        print("... dictionary successfully created.")
        
        return new_dictionary

    except :
        
        print(".. no dictionary data, please connect to server.")

SFD = read_dictionary_file(department_data_name)

Trying to load up dictionary  department_names  ...
... dictionary successfully created.


In [5]:
#This uploads the person data into the program from the file, cleaning the data whilst it does it.

print("Trying to load up data ...")

people = []
meetings = []

try:
    with open(os.getcwd() + '\data\\' + people_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            people.append([clean_text(r[2:-1]) for r in row])
    
    with open(os.getcwd() + '\data\\' + meeting_data_name +'.csv', 'r') as csvfile:
        dump = list(csv.reader(csvfile))
        for row in dump:
            meetings.append([datetime.strptime(row[0],'%Y-%m-%d'),clean_text(row[1][2:-1]),clean_text(row[2][2:-1]),clean_text(row[3][2:-1]),clean_text(row[4][2:-1])])
    
    print("... data successfully uploaded.")
    
except:
        
    print(".. no back up data, please connect to server.")

Trying to load up data ...
... data successfully uploaded.


In [6]:
#Sorts people into 4 categories, Cambridge university people, Other university people, Other Cambirdge people and the rest.

cambridge_other = []
cambridge_university = []
other_university = []
other = []

for person in people:
    affiliation = tokenize(person[2])
    if 'cambridge' in affiliation:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            cambridge_university.append(person)
        else:
            cambridge_other.append(person)
    elif 'cambridgeshire' in affiliation:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            cambridge_university.append(person)
        else:
            cambridge_other.append(person)
    else:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            other_university.append(person)
        else:
            other.append(person)            

In [7]:
#Checks to see if the words appearing in the group name appears in the persons affilliation.
#Input: two strings affil and group.
#Output: Boolean based on if the inportant words from the group name appear in the persons affiliation.

def is_in_group(affil,group):
    affil_tokenized = perpare_organisation(affil)
    for item in perpare_organisation(group):
        if item not in affil_tokenized:
            return False
    return True

#RECCURSIVE FUNCTION
#Used to create a new dictionary which is a copy of the old one with the additional field people and count.
#Input: A dictionary to copy.
#Output: A copy of that dictionary with additional fields.

def create_count(cur_dic):
    count = {}
    count['people'] = []
    count['count'] = 0
    
    for key in cur_dic.keys():
        count[key] = create_count(cur_dic[key])
    
    return count

#RECCURSIVE FUNCTION
#Used to search through out dictionary structure and find all groups which they could fit into.
#Input: A dictionary cur_dic, the persons data and the name of the dictionaries used to get here dic_names.
#Output: List of dictionary locations that would git the person.

def sort_person(cur_dic, person, dic_names):
    keys = cur_dic.keys()
    possibilities = []
    for group in keys:
        new_list = dic_names.copy()
        new_list.append(group)
        if is_in_group(person[2],group):
            possibilities.append(new_list)
        possibilities += sort_person(cur_dic[group], person,new_list)
    return possibilities

#RECCURSIVE FUNCTION
#Adds the person to the dictionary location specified by names.
#Input: A dictionary, the location data of the dictionary, index of how far through you are and the person you want to add.
#Output: Empty.

def add_this(cur_dic, names, index, person):
    if len(names) <= index:
        cur_dic["people"].append(person)
    else:
        add_this(cur_dic[names[index]],names, index + 1, person)
    
#RECCURSIVE FUNCTION
#Sums the number of people in area of the dictionary, including lower layers.
#Input: A dictionary to sum.
#Output: The number of people in this dictionary
    
def sum_dic(cur_dic):
    keys = [key for key in cur_dic.keys() if key not in ['people','count']]
    for key in keys:
        cur_dic['count'] += sum_dic(cur_dic[key])
    cur_dic['count'] += len(cur_dic['people'])
    return cur_dic['count']

#Takes a list of people with their affiliations and sorts them according to the dictionary given.
#Input: List of people with their affiliation in the 3rd entry of a list and a dictionary to sort them by.
#Output: A copy of the given dictionary with additional entries at each layer containing the people that fit and the number of them.

def sort_people(people, sort_dict):
    
    count = create_count(sort_dict)
    
    for person in people:

        possibilities = sort_person(sort_dict, person, [])

        if len(possibilities) == 0:
            count["people"].append(person)
        elif len(possibilities) == 1:
            add_this(count,possibilities[0],0,person)
        else:
            for set_1 in possibilities:
                possibility_1 = set_1[len(set_1) - 1]
                for set_2 in possibilities:
                    possibility_2 = set_2[len(set_2) - 1]
                    if possibility_1 == possibility_2:
                        continue
                    elif is_in_group(possibility_2,possibility_1):
                        possibilities.remove(set_1)
                        break
            if len(possibilities) == 1:
                add_this(count,possibilities[0],0,person)
            else:
                count["people"].append(person)
    
    sum_dic(count)
    
    return count

#Filters the meetins by year.
#Input: List of all meetings, start_date and end_date.
#Output: List of meetings occuring after start_date and before end_date inclusive.

def filter_meetings(meetings, start_date, end_date):
    new_meetings = []
    for meeting in meetings:
        if meeting[0] >= start_date and meeting[0] <= end_date:
            new_meetings.append(meeting)
    
    return new_meetings

In [8]:
#Sorts the people from Cambridge University into departments.

sorted_people = sort_people(cambridge_university, SFD)

#Prints stats about the sorting procedure.

print ("PRINT HOW MANY PEOPLE ARE IN UNACCOUNTED FOR ORGANISATIONS")
unaccount_affiliations = [person[2] for person in sorted_people['people']]
print ('TOTAL = ', len(sorted_people['people']))
for affil in set(unaccount_affiliations):
    print (affil, unaccount_affiliations.count(affil))

total = 0
    
print ("")
print ("HOW MANY ARE IN EACH SCHOOL")
for school in SFD.keys():
    print (school, sorted_people[school]["count"])
    total += sorted_people[school]["count"]

print("")
print('LOCATED AND IDENTIFIED = ', total)
    
print ("")
print ("HOW MANY ARE IN EACH FACULTY")
for school in SFD.keys():
    for faculty in SFD[school].keys():
        print (faculty, sorted_people[school][faculty]["count"])

PRINT HOW MANY PEOPLE ARE IN UNACCOUNTED FOR ORGANISATIONS
TOTAL =  1263
Univ Cambridge 1
Statistical Laboratory, University of Cambridge 13
The Well-being Institute, University of Cambridge 3
Mongolia and Inner Asia Studies Unit (MIASU), University of Cambridge 10
Nanophotonics Centre, Cambridge University 1
Centre of South Asian Studies, University of Cambridge 4
Centre for Entrepreneurial Learning (CfEL), University of Cambridge 5
Africa Together, University of Cambridge 2
Cambridge University Entrepreneurs 1
Cambridge University Government Policy Programme 1
Student, Cambridge University 1
University of Cambridge Alumni 1
Centre for Financial Research, University of Cambridge 2
Cambridge University Hospitals NHS Foundation Trust 29
North West Cambridge Development, University of Cambridge 1
UK Innovation Research Centre, University of Cambridge 1
Gonville and Caius College, University of Cambridge 1
University of Cambridge Computer Lab 1
Centre for Music and Science, Cambridge Univ

In [9]:
academics = []

for meeting in [[meet[3],meet[4]] for meet in meetings]:
    if meeting not in [academic[0] for academic in academics]:
        academics.append([meeting,[person[2] for person in people if person[0] == meeting[0] and person[1] == meeting[1]]])    

cambridge_other = []
cambridge_university = []
colleges = []
other_university = []
other = []

for academic in academics:
    affiliation = [word for affiliation in academic[1] for word in perpare_organisation(affiliation)]
    data_saved = [academic[0][0],academic[0][1],academic[1],affiliation]
    if 'cambridge' in affiliation:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            cambridge_university.append(data_saved)
        elif 'college' in affiliation:
            colleges.append(data_saved)
        else:
            cambridge_other.append(data_saved)
    elif 'cambridgeshire' in affiliation:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            cambridge_university.append(data_saved)
        elif 'college' in affiliation:
            colleges.append(data_saved)
        else:
            cambridge_other.append(data_saved)
    elif 'college' in affiliation:
        colleges.append(data_saved)
    else:
        if 'uni' in affiliation or 'university' in affiliation or 'univ' in affiliation:
            other_university.append(data_saved)
        else:
            other.append(data_saved)


In [10]:
#Sorts a list of people into there prearranged set based upon a mapping given in the dictionary.
#Input: List of people [first_name, second_name, primary_affilitation, ...] and a dictionary 
#'affiliation' -> [primary_tag, secondary_tag].
#Output: List of people with tags attached [first_name, second_name, ... , primary_tag, secondary_tag]

def sort_people(people, sorting_dictionary):
    sorted_people = []
    for person in people:
        new_person = person.copy()
        new_person.append(sorting_dictionary[person[2]][0])
        new_person.append(sorting_dictionary[person[2]][1])
        sorted_people.append()

'''
academics = []

for meeting in [[meet[3],meet[4]] for meet in filter_meetings(meetings,datetime.strptime('2017-01-01','%Y-%m-%d'),datetime.strptime('2018-12-31','%Y-%m-%d'))]:
    if meeting not in [academic[0] for academic in academics]:
        academics.append([meeting,[person[2] for person in people if person[0] == meeting[0] and person[1] == meeting[1]]])

fellows = []

for meeting in [[meet[1],meet[2]] for meet in filter_meetings(meetings,datetime.strptime('2017-01-01','%Y-%m-%d'),datetime.strptime('2018-12-31','%Y-%m-%d'))]:
    if meeting not in [fellow[0] for fellow in fellows]:
        fellows.append([meeting,[person[2] for person in people if person[0] == meeting[0] and person[1] == meeting[1]]])
'''




"\nacademics = []\n\nfor meeting in [[meet[3],meet[4]] for meet in filter_meetings(meetings,datetime.strptime('2017-01-01','%Y-%m-%d'),datetime.strptime('2018-12-31','%Y-%m-%d'))]:\n    if meeting not in [academic[0] for academic in academics]:\n        academics.append([meeting,[person[2] for person in people if person[0] == meeting[0] and person[1] == meeting[1]]])\n\nfellows = []\n\nfor meeting in [[meet[1],meet[2]] for meet in filter_meetings(meetings,datetime.strptime('2017-01-01','%Y-%m-%d'),datetime.strptime('2018-12-31','%Y-%m-%d'))]:\n    if meeting not in [fellow[0] for fellow in fellows]:\n        fellows.append([meeting,[person[2] for person in people if person[0] == meeting[0] and person[1] == meeting[1]]])\n"